In [1]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import talib as ta2
import numpy as np
from datetime import datetime as dt
from datetime import timedelta as delta
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
stock = 'wipro'
start_date = '2000-01-01'
end_date = (dt.now() + delta(1)).strftime('%Y-%m-%d')

df = yf.download(f"{stock}.NS", period='1d', start=start_date, end=end_date)

print(max(df.index))

[*********************100%%**********************]  1 of 1 completed
2023-09-15 00:00:00


In [3]:
df['SMA_10'] = ta.sma(df['Adj Close'],length=10)
df['SMA_30'] = ta.sma(df['Adj Close'],length=30)
df['SMA_50'] = ta.sma(df['Adj Close'],length=50)
df['SMA_200'] = ta.sma(df['Adj Close'],length=200)

macd = ta.macd(df['Adj Close'], fast=12, slow=26, signal=9)
df['MACD'] = macd['MACD_12_26_9']
df['MACD_signal'] = macd['MACDs_12_26_9']
df['MACD_hist'] = macd['MACDh_12_26_9']

In [4]:
df['10_cross_30'] = np.where(df['SMA_10'] > df['SMA_30'], 1, 0)

df['MACD_Signal_MACD'] = np.where(df['MACD_signal'] < df['MACD'], 1, 0)

df['MACD_lim'] = np.where(df['MACD']>0, 1, 0)

df['abv_50'] = np.where((df['SMA_30']>df['SMA_50'])&(df['SMA_10']>df['SMA_50']), 1, 0)

df['abv_200'] = np.where((df['SMA_30']>df['SMA_200'])&(df['SMA_10']>df['SMA_200'])&(df['SMA_50']>df['SMA_200']), 1, 0)

In [5]:
condition_1 = lambda x: ((x['10_cross_30']==1) and (x['MACD_Signal_MACD']==1) and (x['MACD_lim']==0) and (x['abv_50']==1) and (x['abv_200']==0))

condition_2 = lambda x: ((x['10_cross_30']==1) and (x['MACD_Signal_MACD']==0) and (x['MACD_lim']==1) and (x['abv_50']==0) and (x['abv_200']==0))

condition_3 = lambda x: ((x['10_cross_30']==1) and (x['MACD_Signal_MACD']==1) and (x['MACD_lim']==0) and (x['abv_50']==0) and (x['abv_200']==0))

condition_4 = lambda x: ((x['10_cross_30']==1) and (x['MACD_Signal_MACD']==0) and (x['MACD_lim']==0) and (x['abv_50']==0) and (x['abv_200']==1))

condition_5 = lambda x: ((x['10_cross_30']==0) and (x['MACD_Signal_MACD']==1) and (x['MACD_lim']==1) and (x['abv_50']==0) and (x['abv_200']==0))

In [15]:
TRADES = []
trade_in_progress = False
holding_period = 14
check_trend_change = False

In [16]:
def check(row):
    global TRADES, trade_in_progress
    
    if(trade_in_progress):
        _data = TRADES[-1]
        # time to sell after n holding days
        if(row['index']==_data['sell_index']):
            _data['sell_price'] = row['Adj Close']
            _data['sell_date'] = str(pd.to_datetime(row['Date']).date())
            _data['returns'] = round((_data['sell_price']-_data['buy_price'])/_data['buy_price']*100, 2)
            TRADES[-1] = _data
            trade_in_progress = False

        elif((row['index']<_data['sell_index']) and check_trend_change):
            close_trade = False

            # sell if any condition is violated before n holding days
            if((_data['strategy'] in [1]) and ((row['10_cross_30']==0) or (row['MACD_Signal_MACD']==0))): close_trade = True
            elif((_data['strategy']==2) and ((row['10_cross_30']==0) or (row['MACD_lim']==0))): close_trade = True
            elif((_data['strategy']==4) and (row['10_cross_30']==0)): close_trade = True
            elif((_data['strategy']==5) and ((row['MACD_Signal_MACD']==0) or (row['MACD_lim']==0))): close_trade = True

            if(close_trade):
                _data['sell_price'] = row['Adj Close']
                _data['sell_date'] = str(pd.to_datetime(row['Date']).date())
                _data['returns'] = round((_data['sell_price']-_data['buy_price'])/_data['buy_price']*100, 2)
                _data['trend_changed'] = True
                _data['sell_index'] = row['index']
                TRADES[-1] = _data
                trade_in_progress = False
                close_trade = False
            
    else:
        strategy = 0
        if(condition_1(row)): strategy = 1
        elif(condition_2(row)): strategy = 2
        elif(condition_3(row)): strategy = 3
        elif(condition_4(row)): strategy = 4
        elif(condition_5(row)): strategy = 5

        if(strategy>0): 
            trade_in_progress = True
            _data = {
                'strategy': strategy,
                'buy_date': str(pd.to_datetime(row['Date']).date()),
                'buy_index': row['index'],
                'sell_date': '',
                'sell_index': row['index'] + holding_period,
                'buy_price': row['Adj Close'],
                'sell_price': '',
                'returns': 0            
            }
            TRADES.append(_data)
            del _data

df.reset_index().reset_index().apply(check, axis=1)

print('done performing backtesting')

done performing backtesting


In [17]:
TRADES[-3:]

[{'strategy': 5,
  'buy_date': '2023-01-16',
  'buy_index': 5758,
  'sell_date': '2023-02-06',
  'sell_index': 5772,
  'buy_price': 397.6206359863281,
  'sell_price': 404.6499938964844,
  'returns': 1.77},
 {'strategy': 3,
  'buy_date': '2023-04-27',
  'buy_index': 5825,
  'sell_date': '2023-05-18',
  'sell_index': 5839,
  'buy_price': 374.3999938964844,
  'sell_price': 382.6000061035156,
  'returns': 2.19},
 {'strategy': 5,
  'buy_date': '2023-07-06',
  'buy_index': 5873,
  'sell_date': '2023-07-26',
  'sell_index': 5887,
  'buy_price': 396.20001220703125,
  'sell_price': 402.75,
  'returns': 1.65}]

In [18]:
pd.DataFrame(TRADES)['returns'].mean(), pd.DataFrame(TRADES)['returns'].median(), pd.DataFrame(TRADES)['returns'].sum()

(2.60796875, 1.71, 166.91)

In [19]:
pd.DataFrame(TRADES)

,strategy,buy_date,buy_index,sell_date,sell_index,buy_price,sell_price,returns
0,2,2000-02-25,39,2000-03-16,53,210.114075,140.603210,-33.08
1,3,2000-06-15,118,2000-07-05,132,79.048218,97.842346,23.78
2,5,2000-08-22,166,2000-09-11,180,84.691360,99.766747,17.80
3,3,2000-11-10,224,2000-11-30,238,71.847855,78.860069,9.76
4,5,2001-01-10,267,2001-01-30,281,83.094330,86.554550,4.16
...,...,...,...,...,...,...,...,...
59,3,2022-08-02,5645,2022-08-24,5659,428.544464,415.127502,-3.13
60,3,2022-11-14,5713,2022-12-02,5727,395.575653,411.386719,4.00
61,5,2023-01-16,5758,2023-02-06,5772,397.620636,404.649994,1.77
62,3,2023-04-27,5825,2023-05-18,5839,374.399994,382.600006,2.19


In [20]:
df.tail(5)

,Open,High,Low,Close,Adj Close,Volume,SMA_10,SMA_30,SMA_50,SMA_200,MACD,MACD_signal,MACD_hist,10_cross_30,MACD_Signal_MACD,MACD_lim,abv_50,abv_200
Date,,,,,,,,,,,,,,,,,,
2023-09-11,433.000000,437.500000,430.100006,434.850006,434.850006,4126657,423.300003,415.791667,409.982001,394.620744,6.410385,4.621355,1.789030,1,1,1,1,1
2023-09-12,437.100006,441.500000,426.899994,435.000000,435.000000,7272664,425.990002,416.790000,410.850001,394.856770,6.763082,5.049700,1.713382,1,1,1,1,1
2023-09-13,434.950012,437.950012,431.500000,435.700012,435.700012,3722835,428.840002,417.748334,411.645001,395.048913,7.018180,5.443396,1.574784,1,1,1,1,1
2023-09-14,438.399994,443.250000,435.299988,436.450012,436.450012,7447505,431.645004,418.870001,412.496001,395.222111,7.197894,5.794296,1.403598,1,1,1,1,1
2023-09-15,439.799988,443.750000,438.250000,441.049988,441.049988,9917938,434.115002,420.250001,413.393000,395.404095,7.623617,6.160160,1.463457,1,1,1,1,1
